In [1]:
from os.path import abspath
from pyspark.sql import SparkSession
import pyspark.sql.functions as f


# set default location for [spark-warehouse]
warehouse_location = abspath('spark-warehouse')

In [2]:
spark = (
    SparkSession
    .builder
    .appName("Delta Lake")
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.1.1,io.delta:delta-core_2.12:1.0.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.debug.maxToStringFields","1000")
    .config("spark.kryoserializer.buffer.max","128m")
    .enableHiveSupport()
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-core_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-276658d1-8e6c-47b3-b332-87c9d8f85644;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.1.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.1 in central
	found org.apache.spark#spark-token

In [3]:
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.project.id", "hering-datalake-prod")
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

---

In [4]:
def start_stream (topic, landing_path, kafka_bootstrap):
    return (
        spark
        .readStream
        .format('kafka')
        .option('kafka.bootstrap.servers', kafka_bootstrap)
        .option('subscribe', topic)
        .option('startingOffsets', 'earliest')
        .option("failOnDataLoss", "false")
        .load()
        .withColumn('year', f.date_format(f.col('timestamp'), 'yyyy'))
        .withColumn('month', f.date_format(f.col('timestamp'), 'MM'))
        .withColumn('day', f.date_format(f.col('timestamp'), 'dd'))
        .withColumn('hour', f.date_format(f.col('timestamp'), 'HH'))
        .writeStream
        .partitionBy('year', 'month', 'day', 'hour')
        .format('json')
        .option('checkpointLocation', landing_path + 'checkpoint')
        .start(landing_path)
    )

In [5]:
def start_stream (topic, landing_path, kafka_bootstrap):
    return (
        spark
        .read
        .format('kafka')
        .option('kafka.bootstrap.servers', kafka_bootstrap)
        .option('subscribe', topic)
        .option('startingOffsets', 'earliest')
        .option("failOnDataLoss", "false")
        .load()
        .limit(10000000)
        .withColumn('year', f.date_format(f.col('timestamp'), 'yyyy'))
        .withColumn('month', f.date_format(f.col('timestamp'), 'MM'))
        .withColumn('day', f.date_format(f.col('timestamp'), 'dd'))
        .withColumn('hour', f.date_format(f.col('timestamp'), 'HH'))
        .write
        .partitionBy('year', 'month', 'day', 'hour')
        .format('json')
        .option('checkpointLocation', landing_path + 'checkpoint')
        .save(landing_path)
    )

In [8]:
kafka_bootstrap = '172.31.157.31:9094'
topics_list =  'bnsvher327.prcf.PRCF.CFMVTFAC'
landing_path_base = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher361/db_analise/crm/'


In [11]:
kafka_bootstrap = 'edh-kafka-bootstrap.ingestion.svc:9092'
topics_list =  ['eih.bnsvher361.db_analise.crm.tb_base_clientes']
landing_path_base = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher361/db_analise/crm/'


In [9]:
df = (
    spark
    .read
    .format('kafka')
    .option('kafka.bootstrap.servers', kafka_bootstrap)
    .option('subscribe', topics_list)
    .option('startingOffsets', 'earliest')
    .load()
)

In [ ]:
df.limit(10).toPandas()

22/06/07 19:43:14 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-relation-d6d1439c-aed8-4671-9d65-1c02e054d3dd-driver-0-1, groupId=spark-kafka-relation-d6d1439c-aed8-4671-9d65-1c02e054d3dd-driver-0] Connection to node -1 (/172.31.157.31:9094) could not be established. Broker may not be available.
22/06/07 19:43:14 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-relation-d6d1439c-aed8-4671-9d65-1c02e054d3dd-driver-0-1, groupId=spark-kafka-relation-d6d1439c-aed8-4671-9d65-1c02e054d3dd-driver-0] Bootstrap broker 172.31.157.31:9094 (id: -1 rack: null) disconnected
22/06/07 19:45:25 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-relation-d6d1439c-aed8-4671-9d65-1c02e054d3dd-driver-0-1, groupId=spark-kafka-relation-d6d1439c-aed8-4671-9d65-1c02e054d3dd-driver-0] Connection to node -1 (/172.31.157.31:9094) could not be established. Broker may not be available.
22/06/07 19:45:25 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-relation-d6d1439c-a

22/06/07 20:13:49 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-relation-d6d1439c-aed8-4671-9d65-1c02e054d3dd-driver-0-1, groupId=spark-kafka-relation-d6d1439c-aed8-4671-9d65-1c02e054d3dd-driver-0] Connection to node -1 (/172.31.157.31:9094) could not be established. Broker may not be available.
22/06/07 20:13:49 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-relation-d6d1439c-aed8-4671-9d65-1c02e054d3dd-driver-0-1, groupId=spark-kafka-relation-d6d1439c-aed8-4671-9d65-1c02e054d3dd-driver-0] Bootstrap broker 172.31.157.31:9094 (id: -1 rack: null) disconnected
22/06/07 20:16:00 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-relation-d6d1439c-aed8-4671-9d65-1c02e054d3dd-driver-0-1, groupId=spark-kafka-relation-d6d1439c-aed8-4671-9d65-1c02e054d3dd-driver-0] Connection to node -1 (/172.31.157.31:9094) could not be established. Broker may not be available.
22/06/07 20:16:00 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-relation-d6d1439c-a

In [15]:
stream_list = []
for topic in topics_list:
    landing_path = landing_path_base + topic.split('.')[-1].lower() + '/'
    stream_list.append(start_stream(topic, landing_path, kafka_bootstrap))

22/06/07 19:36:55 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-relation-cb224c6f-9967-4d0f-b38e-552612a6116f-driver-0-1, groupId=spark-kafka-relation-cb224c6f-9967-4d0f-b38e-552612a6116f-driver-0] Connection to node -1 (/35.232.184.19:9094) could not be established. Broker may not be available.
22/06/07 19:36:55 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-relation-cb224c6f-9967-4d0f-b38e-552612a6116f-driver-0-1, groupId=spark-kafka-relation-cb224c6f-9967-4d0f-b38e-552612a6116f-driver-0] Bootstrap broker 35.232.184.19:9094 (id: -1 rack: null) disconnected
22/06/07 19:36:55 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-relation-cb224c6f-9967-4d0f-b38e-552612a6116f-driver-0-1, groupId=spark-kafka-relation-cb224c6f-9967-4d0f-b38e-552612a6116f-driver-0] Connection to node -1 (/35.232.184.19:9094) could not be established. Broker may not be available.
22/06/07 19:36:55 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-relation-cb224c6f-9

22/06/07 19:37:08 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-relation-cb224c6f-9967-4d0f-b38e-552612a6116f-driver-0-1, groupId=spark-kafka-relation-cb224c6f-9967-4d0f-b38e-552612a6116f-driver-0] Connection to node -1 (/35.232.184.19:9094) could not be established. Broker may not be available.
22/06/07 19:37:08 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-relation-cb224c6f-9967-4d0f-b38e-552612a6116f-driver-0-1, groupId=spark-kafka-relation-cb224c6f-9967-4d0f-b38e-552612a6116f-driver-0] Bootstrap broker 35.232.184.19:9094 (id: -1 rack: null) disconnected
22/06/07 19:37:09 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-relation-cb224c6f-9967-4d0f-b38e-552612a6116f-driver-0-1, groupId=spark-kafka-relation-cb224c6f-9967-4d0f-b38e-552612a6116f-driver-0] Connection to node -1 (/35.232.184.19:9094) could not be established. Broker may not be available.
22/06/07 19:37:09 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-relation-cb224c6f-9

KeyboardInterrupt: 

In [ ]:
stream_list[0].awaitTermination()

22/05/23 09:45:20 WARN TaskMemoryManager: Failed to allocate a page (8388608 bytes), try again.
22/05/23 09:45:20 WARN TaskMemoryManager: Failed to allocate a page (8388608 bytes), try again.
22/05/23 09:45:23 ERROR Executor: Exception in task 1.0 in stage 0.0 (TID 1)
java.lang.OutOfMemoryError: Java heap space
22/05/23 09:45:23 ERROR SparkUncaughtExceptionHandler: Uncaught exception in thread Thread[Executor task launch worker for task 1.0 in stage 0.0 (TID 1),5,main]
java.lang.OutOfMemoryError: Java heap space
22/05/23 09:45:23 ERROR Inbox: Ignoring error                       (0 + 3) / 3]
java.util.concurrent.RejectedExecutionException: Task org.apache.spark.executor.Executor$TaskRunner@258eae0e rejected from java.util.concurrent.ThreadPoolExecutor@72720885[Shutting down, pool size = 2, active threads = 2, queued tasks = 0, completed tasks = 0]
	at java.base/java.util.concurrent.ThreadPoolExecutor$AbortPolicy.rejectedExecution(ThreadPoolExecutor.java:2055)
	at java.base/java.util.co

StreamingQueryException: Job aborted.
=== Streaming Query ===
Identifier: [id = 1f2670ff-2eb7-420d-b8fd-d530c730fce9, runId = c38e9fcd-2410-4de0-a707-2dbadfe5e2fc]
Current Committed Offsets: {}
Current Available Offsets: {KafkaV2[Subscribe[eih.bnsvher361.db_analise.crm.tb_base_clientes]]: {"eih.bnsvher361.db_analise.crm.tb_base_clientes":{"2":37091211,"1":37250623,"0":37137381}}}

Current State: ACTIVE
Thread State: RUNNABLE

Logical Plan:
Project [key#7, value#8, topic#9, partition#10, offset#11L, timestamp#12, timestampType#13, year#21, month#30, day#40, date_format(timestamp#12, HH, Some(Etc/UTC)) AS hour#51]
+- Project [key#7, value#8, topic#9, partition#10, offset#11L, timestamp#12, timestampType#13, year#21, month#30, date_format(timestamp#12, dd, Some(Etc/UTC)) AS day#40]
   +- Project [key#7, value#8, topic#9, partition#10, offset#11L, timestamp#12, timestampType#13, year#21, date_format(timestamp#12, MM, Some(Etc/UTC)) AS month#30]
      +- Project [key#7, value#8, topic#9, pa

22/05/23 09:45:25 ERROR TaskMemoryManager: error while calling spill() on org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter@7359676d
java.io.FileNotFoundException: /tmp/blockmgr-69b17966-4b14-491e-a960-025b27d451a0/06/temp_local_0b0c30f8-8a52-4c11-aa6f-6243fdc30599 (No such file or directory)
	at java.base/java.io.FileOutputStream.open0(Native Method)
	at java.base/java.io.FileOutputStream.open(FileOutputStream.java:298)
	at java.base/java.io.FileOutputStream.<init>(FileOutputStream.java:237)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:105)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:118)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:257)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeSorterSpillWriter.<init>(UnsafeSorterSpillWriter.java:81)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.spill(UnsafeExternalSorter.ja